<a href="https://colab.research.google.com/github/gcivitate/modelo_titanic/blob/main/C%C3%B3pia_de_JK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import glob
import pandas as pd
import numpy
import scipy
import statsmodels
import sklearn
from datetime import date, time, datetime, timedelta
from dateutil.relativedelta import relativedelta
from pandas.core.groupby import NamedAgg

In [ ]:
# Ler os arquivos report_ que terminam com *xlsx
def load_report():
    path = '/content'
    test = pd.DataFrame()
    filenames = glob.glob(path + "/*.xlsx")


    for df in filenames:
        df_report = pd.read_excel(df)
        test = test.append(df_report,ignore_index = True)
    return test

def load_view():
  """
  Função que lê importa o arquivo exportado da view v_dados_cli_maq_jkcontrol
  """
  df = pd.read_csv("/content/v_dados_cli_maq_jkcontrol.csv", encoding="ISO-8859-1")
  return df

def load_maquina():
  """
  Função que lê o arquivo máquina operador
  """
  df = pd.read_csv("/content/maquina_01_device_rfid_variable_Gs5M.csv")
  return df

def treatment_maquina(df_maquina):
  """
  Função que faz o tratamento do dataframe do df_maquina.
  """
  # Trata o Json dentro da coluna context_rfid
  values = df_maquina.context_rfid.str.replace("'", '"').apply(json.loads).tolist()
  df = pd.DataFrame(values)

  df = pd.concat([df, df_maquina],axis = 1, join = 'outer', ignore_index=False, sort=False)

  df = df.drop('context_rfid', axis=1)

  # Separa o código do nome do funcionario
  df['cod_fun'] = df['cod_fun'].str.replace('-','')

  df_func = df['cod_fun'].str.split('(\d+)([A-Za-z]+)', expand=True)
  df_func = df_func.loc[:,[1,2]]
  df_func.rename(columns={1:'cod_func', 2:'name_func'}, inplace=True)

  df = df.drop('cod_fun', axis=1)

  df = pd.concat([df_func, df],axis = 1, join = 'outer', ignore_index=False, sort=False)

  # Renomear a coluna date
  df = df.rename(columns={'date (America/Sao_Paulo)': 'date'})

  return df

df_view = load_view()
df_report = load_report()
df = load_maquina()
df_maquina = treatment_maquina(df)


,cod_func,name_func,cod_prd,cod_maq,cod_pdr,cod_set,timestamp,rfid,date
0,103,MATEUS,NaN,NaN,NaN,NaN,1659454791242,1.0,2022-08-02 12:39:51.242000-03:00
1,26,DINHA,NaN,NaN,NaN,NaN,1659454787009,1.0,2022-08-02 12:39:47.009000-03:00
2,26,DINHA,NaN,NaN,NaN,NaN,1659451063772,1.0,2022-08-02 11:37:43.772000-03:00
3,103,MATEUS,NaN,NaN,NaN,NaN,1659450990030,1.0,2022-08-02 11:36:30.030000-03:00
4,103,MATEUS,NaN,NaN,NaN,NaN,1659431247784,1.0,2022-08-02 06:07:27.784000-03:00
...,...,...,...,...,...,...,...,...,...
624,NaN,NaN,NaN,1,NaN,NaN,1651769434321,1.0,2022-05-05 13:50:34.321000-03:00
625,NaN,NaN,NaN,NaN,NaN,0.0,1651756638475,1.0,2022-05-05 10:17:18.475000-03:00
626,NaN,NaN,NaN,160,NaN,NaN,1651756509741,1.0,2022-05-05 10:15:09.741000-03:00
627,NaN,NaN,12345678910111,NaN,NaN,NaN,1651618101701,1.0,2022-05-03 19:48:21.701000-03:00


In [ ]:
# Análise descritiva:

#Verificando quantidade por tipo de parada:

count_maq = df_maquina.groupby('cod_maq') \
            .agg(n = pd.NamedAgg('cod_maq', 'count')) \
            .sort_values(by='n')

count_maq

In [ ]:
#Verificando quantidade de parada por funcionario
count_func = df_maquina.groupby('cod_func') \
            .agg(n = pd.NamedAgg('cod_func', 'count')) \
            .sort_values(by='n')

count_func

In [ ]:
#Verificando quantidade de produtos
count_prd = df_maquina.groupby('cod_prd') \
            .agg(n = pd.NamedAgg('cod_prd', 'count')) \
            .sort_values(by='n')

count_prd

In [ ]:
# Media de produção:

df_report.groupby('Date')['ciclo (maquina_01 chen hsong)'] \
          .apply(lambda x: x.mode().iloc[0]) \
          .to_frame() \
          .sort_values(by='Date') \
          .reset_index()




In [ ]:
# Media ciclo maquina 01 em segundos
media_df_report = (df_report['ciclo (maquina_01 chen hsong)'].mean()) / 1000

print(f"Media do ciclo total da maquina chen hsong: {media_df_report}")

In [ ]:
# Desvio médio absoluto


df_report.groupby('Date') \
          .agg(mad_ciclo = pd.NamedAgg('ciclo (maquina_01 chen hsong)', 'mad'))


In [ ]:
# Peças com defeito

pc_defeito = df_report['peca_defeito (maquina_01 chen hsong)'].sum()
print(f"Número de peças produziadas com defeit: {pc_defeito}")

Número de peças produziadas com defeit: 153.0


In [ ]:
#Groupby por Data e maquina01
#df_report.sort_values(by='Date', inplace=True)
#df_report.loc[:, ['Date', 'ciclo (maquina_01 chen hsong)']]

